Importing the necessary packages

In [ ]:
import torch
import pandas as pd
from torchvision import models
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Loading the dataset. <br>
The dataset is a csv file with one column containing class label and the others containing pixel-intensity values taken in row-wise order.

In [ ]:
path = '/content/drive/MyDrive/sign_mnist/sign_mnist_train.csv'
width, height = 28, 28
image_size = (width, height)
data = pd.read_csv(path)
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,156,158,160,163,165,159,166,168,170,170,171,171,171,172,171,171,170,170,169,111,121,129,135,141,144,148,151,154,157,160,...,205,206,206,207,207,206,206,204,205,204,203,202,142,151,160,172,196,188,188,190,135,96,86,77,77,79,176,205,207,207,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,157,158,156,154,154,153,152,151,149,149,148,147,146,144,142,143,138,92,108,158,159,159,159,160,160,160,160,160,160,160,...,100,78,120,157,168,107,99,121,133,97,95,120,135,116,95,79,69,86,139,173,200,185,175,198,124,118,94,140,133,84,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,186,185,185,185,184,184,184,181,181,179,179,179,178,178,109,52,66,77,83,188,189,189,188,188,189,188,188,188,188,187,...,203,204,203,201,200,200,199,198,196,195,194,193,198,166,132,114,89,74,79,77,74,78,132,188,210,209,206,205,204,203,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,211,209,207,208,207,206,203,202,201,200,198,197,195,192,197,171,51,52,54,212,213,215,215,212,212,213,212,212,211,211,...,247,242,233,231,230,229,227,225,223,221,220,216,58,51,49,50,57,60,17,15,18,17,19,1,159,255,237,239,237,236,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,186,188,189,189,190,191,189,190,190,187,190,192,193,191,191,192,192,194,194,166,169,172,174,177,180,182,185,186,187,190,...,90,77,88,117,123,127,129,134,145,152,156,179,105,106,105,104,104,104,175,199,178,152,136,130,136,150,118,92,85,76,92,105,105,108,133,163,157,163,164,179


Loading the pre-trained model.

In [ ]:
densenet = models.densenet161(pretrained = True)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


  0%|          | 0.00/110M [00:00<?, ?B/s]

Freezing the parameters.

In [ ]:
for param in densenet.parameters():
  param.requires_grad = False

In [ ]:
print(densenet)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [ ]:
print(densenet.classifier)

Linear(in_features=2208, out_features=1000, bias=True)


In [ ]:
num_classes = 25
hidden_dim = [1000, 500]

Attaching classifier layers to the end of the model.

In [ ]:
final_in_features = densenet.classifier.in_features
densenet.classifier = nn.Sequential(nn.Linear(final_in_features, hidden_dim[0]),
                                    nn.Sigmoid(),
                                    nn.Linear(hidden_dim[0], hidden_dim[1]),
                                    nn.Sigmoid(),
                                    nn.Linear(hidden_dim[1], num_classes))

In [ ]:
densenet

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

Ensuring that only the parameters corresponding to the layers we added are trainable.

In [ ]:
for param in densenet.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([1000, 2208])
torch.Size([1000])
torch.Size([500, 1000])
torch.Size([500])
torch.Size([25, 500])
torch.Size([25])


Setting the learning rate and batch-size.

In [ ]:
lr = 0.01
batch_size = 128

Loading previously trained model (optional).

In [ ]:
path = '/content/drive/MyDrive/sign_mnist/saved_models/densenet3'
densenet.load_state_dict(torch.load(path))

<All keys matched successfully>

Defining loss function, optimizer and moving the model to GPU for faster training.

In [ ]:
densenet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(densenet.parameters(), lr = lr)

Creating custom dataset class <br>
As a subtle pre-processing step of sorts, padding is added to the images. The reason for this is that the model requires a minimum input image size of 29x29 while our images just fall short at 28x28. So, one layer of padding is added (the edge pixels are simply repeated once to extend on all sides by 1 pixel) making the images 30x30 in size which is suitable for the model.

In [ ]:
class SignDataset(Dataset):
  def __init__(self, images, labels):
    self.images = images
    self.labels = labels

  def __len__(self):
    return len(self.labels)
  def __getitem__(self, idx):
    label = self.labels[idx]
    image = self.images[idx].reshape((28, 28))
    image = np.pad(image, 1, mode = 'edge')
    image = image.reshape((1, -1))
    sample = {'Image': image, 'Label' : label}
    return sample

Creating instance of the class with train data

In [ ]:
images = data.values[:, 1: 785]
labels = data.values[:, 0]
train_sd = SignDataset(images, labels)

Using the class object, dataloader is created which will return samples with their corresponding labels in batches.

In [ ]:
train_loader = DataLoader(train_sd, batch_size = batch_size, shuffle = True)

In [ ]:
num_epochs = 5

The training loop. <br>
The images are greyscale originally but the model works only on 3-channel images so two copies of the single channel of the input images is made before passing to the model. Loss is then calculated and parameter update is performed. After each epoch, train accuracy is calculated and displayed for observation.

In [ ]:
for epoch in range(num_epochs):
    for idx, batch in enumerate(train_loader):
        images = batch['Image'].view(-1, 30, 30)
        images = np.repeat(images[..., np.newaxis], 3, -1)
        images = images.reshape(-1, 3, 30, 30)
        images = images.type(torch.FloatTensor).requires_grad_()
    
        labels = batch['Label'].type(torch.LongTensor)
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = densenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    correct = 0
    total = 0

    for idx, batch in enumerate(train_loader):
      images = batch['Image'].view(-1, 30, 30)
      images = np.repeat(images[..., np.newaxis], 3, -1)
      images = images.reshape(-1, 3, 30, 30)
      images = images.type(torch.FloatTensor)

      labels = batch['Label'].type(torch.LongTensor)
      images = images.to(device)
      labels = labels.to(device)
      outputs = densenet(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum()

    accuracy = 100 * correct / total
    print(f'Epoch: {epoch}, Accuracy: {accuracy: .2f}')

Epoch: 0, Accuracy:  87.06
Epoch: 1, Accuracy:  87.22
Epoch: 2, Accuracy:  87.73
Epoch: 3, Accuracy:  87.55
Epoch: 4, Accuracy:  87.71


Saving the model once satisfactory performance level is reached.

In [ ]:
path = '/content/drive/MyDrive/sign_mnist/saved_models/densenet3'
torch.save(densenet.state_dict(), path)

In [ ]:
path = '/content/drive/MyDrive/sign_mnist/sign_mnist_test.csv'
test_data = pd.read_csv(path)

In [ ]:
test_images = test_data.values[:, 1: 785]
test_labels = test_data.values[:, 0]
test_sd = SignDataset(test_images, test_labels)

In [ ]:
test_loader = DataLoader(test_sd, batch_size = batch_size, shuffle = False)

Same process as for the train data is repeated to load the test data and get it ready for model evaluation. Then, test accuracy is calculated and displayed.

In [ ]:
correct = 0
total = 0

for idx, batch in enumerate(test_loader):
  images = batch['Image'].view(-1, 30, 30)
  images = np.repeat(images[..., np.newaxis], 3, -1)
  images = images.reshape(-1, 3, 30, 30)
  images = images.type(torch.FloatTensor)

  labels = batch['Label'].type(torch.LongTensor)
  images = images.to(device)
  labels = labels.to(device)
  outputs = densenet(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy: .2f}')

Test Accuracy:  44.79


Verdict: Despite training with some degree of rigor, the test accuracy simply saturates at some point and refuses to climb any further. Perhaps, the model architecture is to blame and it is simply not a fitting choice for this dataset. 